In [1]:
import pandas as pd
import numpy as np

In [13]:
#Reading the reward points
file_path = r'C:\Users\zawadi\OneDrive - Kim-Fay East Africa Ltd\Documents\Mentorship_Sessions_Clean_Data.csv'
Mentorship_data = pd.read_csv(file_path)
Mentorship_data.head()

,Mentee_id,Mentor_ID,Mentor_Name,Mentee_Name,Session_Number,Session_Duration_Min,Job_Info_Completed,Session_Date,Points_Awarded
0,2001,1003,Sarah Clark,Bob Brown,1,40,Yes,2023-01-01,NaN
1,2002,1003,Emily Davis,Carol White,2,30,Yes,2023-01-08,NaN
2,2003,1005,James Wilson,Jane Smith,2,40,Yes,2023-01-15,NaN
3,2001,1004,Emily Davis,Bob Brown,1,30,No,2023-01-29,NaN
4,2003,1002,David Thompson,Jane Smith,2,50,No,2023-02-05,NaN


In [15]:
## Assigning 250 points for each unique mentor who signed up
Mentorship_data['signup_points'] = 0
unique_mentors = Mentorship_data['Mentor_ID'].unique()
mentor_points = pd.DataFrame(unique_mentors, columns=['Mentor_ID'])
mentor_points['signup_points'] = 250  # One-time points for signing u

In [17]:
Mentorship_data.head()

,Mentee_id,Mentor_ID,Mentor_Name,Mentee_Name,Session_Number,Session_Duration_Min,Job_Info_Completed,Session_Date,Points_Awarded,signup_points
0,2001,1003,Sarah Clark,Bob Brown,1,40,Yes,2023-01-01,NaN,0
1,2002,1003,Emily Davis,Carol White,2,30,Yes,2023-01-08,NaN,0
2,2003,1005,James Wilson,Jane Smith,2,40,Yes,2023-01-15,NaN,0
3,2001,1004,Emily Davis,Bob Brown,1,30,No,2023-01-29,NaN,0
4,2003,1002,David Thompson,Jane Smith,2,50,No,2023-02-05,NaN,0


In [19]:
mentor_points['signup_points']

0    250
1    250
2    250
3    250
4    250
Name: signup_points, dtype: int64

In [27]:
#Allocating points for Mentorship with Two Different Mentees
## Checking if a mentor has mentored at least two different mentees
mentees_per_mentor = Mentorship_data.groupby('Mentor_ID')['Mentee_id'].nunique().reset_index()
mentees_per_mentor.columns = ['Mentor_ID', 'num_of_mentees']

# Allocate 1000 points if they have 2 different mentees
mentees_per_mentor['mentee_points'] = np.where(mentees_per_mentor['num_of_mentees'] >= 2, 1000, 0)



In [31]:
mentees_per_mentor['mentee_points']

0    1000
1    1000
2    1000
3    1000
4    1000
Name: mentee_points, dtype: int32

In [39]:
Mentorship_data.columns = Mentorship_data.columns.str.strip()

In [41]:
Mentorship_data.columns

Index(['Mentee_id', 'Mentor_ID', 'Mentor_Name', 'Mentee_Name',
       'Session_Number', 'Session_Duration_Min', 'Job_Info_Completed',
       'Session_Date', 'Points_Awarded', 'signup_points', 'session_points'],
      dtype='object')

In [45]:
#Allocating points for Mentorship Relationships (500 Points)
# Calculate points per mentor-mentee pair
Mentorship_data['session_points'] = np.where(Mentorship_data['Session_Duration_Min'] >= 30, 250, 0)  # 250 points per session if 30+ mins

In [48]:
# Group by mentor and mentee to find the number of sessions and the sum of session points
relationship_points = Mentorship_data.groupby(['Mentor_ID', 'Mentee_id']).agg(
    num_sessions=('Session_Duration_Min', 'count'),
    total_session_points=('session_points', 'sum'),
    job_info_completed=('Job_Info_Completed', 'max')  # Check if at least one session had job info completed
).reset_index()


In [82]:
# Allocate maximum 500 points if two sessions are conducted with the same mentee and job info is completed
relationship_points['relationship_points'] = np.where(
    (relationship_points['num_sessions'] >= 2) & (relationship_points['job_info_completed'] == True), 
    500, relationship_points['total_session_points']
)

In [84]:
#Combining the results
# Merge mentor signup points
final_points = pd.merge(mentor_points[['Mentor_ID', 'signup_points']], mentees_per_mentor[['Mentor_ID', 'mentee_points']], on='Mentor_ID', how='left')

# Sum the relationship points per mentor
relationship_points_summary = relationship_points.groupby('Mentor_ID')['relationship_points'].sum().reset_index()

# Merge relationship points with the final DataFrame
final_points = pd.merge(final_points, relationship_points_summary[['Mentor_ID', 'relationship_points']], on='Mentor_ID', how='left')

# Fill NaN values with 0 (in case any mentor has no relationship points)
final_points = final_points.fillna(0)

# Calculate the total points for each mentor
final_points['total_points'] = final_points['signup_points'] + final_points['mentee_points'] + final_points['relationship_points']

print(final_points)


   Mentor_ID  signup_points  mentee_points  relationship_points  total_points
0       1003            250           1000                 3500          4750
1       1005            250           1000                 5250          6500
2       1004            250           1000                 4750          6000
3       1002            250           1000                 2250          3500
4       1001            250           1000                 2750          4000
